<h1 style='color: green; font-size: 36px; font-weight: bold;'>Análise de Dados - Pressão Arterial</h1>

## Importando bibliotecas


In [1]:
!pip3 install scipy==1.7 --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 28.5 MB 1.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [2]:
!pip3 install statsmodels==0.12.2 --upgrade 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.set_option('display.max_rows', 50)
#pd.set_option('display.max_columns', 100)
import numpy as np


import statsmodels.api as sm
import seaborn as sns #library especializada em visualização de dados estatísticos e baseada em matplotlib().

import scipy.stats as stats
from scipy.stats import normaltest #A função *normaltest* testa a hipótese nula $H_0$ de que a amostra é proveniente de uma distribuição normal.

## Bibliotecas opcionais


In [4]:
import warnings

warnings.filterwarnings('ignore') # ou warnings.filterwarnings(action='once')

In [5]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 14.5 MB/s 
     |████████████████████████████████| 1.6 MB 7.9 MB/s 


In [7]:
!pip install plotnine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# <font color='red' style='font-size: 30px;'>CONHECENDO O DATASET</font>
<hr style='border: 2px solid red;'>

## O Dataset e o Projeto
<hr>

### Fonte: 
<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Dados coletados pela Prof. Letícia Helena Januário</p>

### Descrição:

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>
O objetivo do trabalho é identificar relação entre os valores pressóricos mensurados em indivíduos com a mesma circunferência de braço, mas com diferentes frações entre os componentes corporais do braço em adultos jovens saudáveis.</p>

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Será estimado um modelo de <b>Machine Learning</b> utilizando a técnica de <b>Regressão Linear</b> para demonstrar os impactos das variáveis disponibilizadas neste dataset sobre os valores pressóricos (Y). No final do projeto teremos um modelo de previsão para o valores médios de pressão segundo os inputs de um conjunto de variáveis (X's).</p>

 ### Dados:
<ul style='font-size: 16px; line-height: 2; text-align: justify;'>
    <li><b> Comprimento</b> - float</li>
    <li><b> Circunferência</b> - float</li>
    <li><b> Tríceps</b> - float</li>
    <li><b> Circunferência Muscular</b> CMB (cm) = CB – (𝜋 x DCT) </li>
    <li><b> Índice de Gordura</b> IGB = (AGB/ATB) x 100.  </li>
     <li><b> PAS</b> - float</li>
    <li><b> PAD</b> - float</li>

</ul> 


#### <font color='red'>Observação</font>
***
> Os seguintes tratamentos foram realizados nos dados originais:

>1. Foram eliminados por informarem ser hipertensos e/ou diabéticos, e/ou portadores de cardiopatias;
>2. Foram eliminados por informarem fazer uso de medicamentos anti-hipertensivos
>3. Foram eliminados 17 por terem a idade menor que 18 ou maior que 29 anos,
>4. Foram eliminados 11 por estarem com os valores pressóricos elevados no momento da coleta de dados: maior ou igual a 140/90 mmHg

# <font color='red' style='font-size: 30px;'>FORMATAÇÃO E LIMPEZA DOS DADOS</font>
<hr style='border: 2px solid red;'>

In [8]:
dados = pd.read_csv('/content/drive/MyDrive/DOUTORADO/3 PUBLICAÇÕES/Artigo Journal/Colab Notebooks/Dados/planilha_completa.csv',  sep=';')

FileNotFoundError: ignored

In [ ]:
dados.head()

In [ ]:
dados.columns

In [ ]:
dados.columns = ['ID', 'Sexo', 'Idade', 'Raca', 'Peso', 'Estatura', 'IMC',
       'Comprimento BD', 'Comprimento BD Seg', 'Circunferencia BD',
       'Triceps BD', 'Circunferencia Muscular BD', 'Area Gordurosa BD',
       'Area Total BD', 'Indice de Gordura BD', 'Comprimento BE',
       'Comprimento BE Seg', 'Circunferencia BE', 'Triceps BE',
       'Circunferencia Muscular BE', 'Area Muscular BE', 'Area Gordurosa BE',
       'Area Total BE', 'Indice de Gordura BE', 'PAS BD', 'PAD BD', 'PAS BE',
       'PAD BE']

In [ ]:
dados.drop(columns=['Estatura', 'Peso', 'Comprimento BD', 'Comprimento BE','Area Muscular BE', 'Area Gordurosa BE','Area Gordurosa BD','Area Total BD', 'Area Total BE'], inplace=True)  
dados.head()

In [ ]:
dados_bd = dados[["ID", 'Idade', 'IMC', 'Sexo','Comprimento BD Seg', 'Circunferencia BD',  'Triceps BD', 'Circunferencia Muscular BD', 'Indice de Gordura BD', 'PAD BD','PAS BD']]
dados_be = dados[["ID", 'Idade', 'IMC', 'Sexo','Comprimento BE Seg', 'Circunferencia BE', 'Triceps BE', 'Circunferencia Muscular BE', 'Indice de Gordura BE','PAD BE','PAS BE']]


In [ ]:
dados_bd.columns = ["ID", 'Idade', 'IMC', 'Sexo', 'Comprimento', 'Circunferência', 'Tríceps', 'Circunferência Muscular', 'Índice de Gordura','PAD', 'PAS']
dados_be.columns = ["ID", 'Idade', 'IMC', 'Sexo', 'Comprimento', 'Circunferência', 'Tríceps', 'Circunferência Muscular', 'Índice de Gordura','PAD', 'PAS']
dados_dobrados = pd.concat([dados_bd, dados_be])
dados_dobrados.index = list(range(1,dados_dobrados.shape[0]+1))

In [ ]:
dados_dobrados.to_csv("dados_dobrados.csv", index=False)
dados_dobrados.to_parquet("dados_dobrados.parquet")

## <font color='red' style='font-size: 25px;'>TABELAS FINAIS</font>
<hr style='border: 2px solid red;'>

### Dados braço esquerdo

In [ ]:
dados_be.head()

### Dados braço direito

In [ ]:
dados_bd.head()

### Dados dobrados

In [ ]:
dados_dobrados.head()

In [ ]:
dados_dobrados.shape

In [ ]:
dados_be.to_parquet("dados_be.parquet")
dados_bd.to_parquet("dados_bd.parquet")

# <font color='red' style='font-size: 30px;'>ANÁLISES DESCRITIVAS DOS DADOS POR HEMICORPOS</font>
<hr style='border: 2px solid red;'>

In [ ]:
dados_bd.describe().round(2)

In [ ]:
dados_be.describe().round(2)

In [ ]:
frequencia_sexo= dados['Sexo'].value_counts()
percentual_sexo = dados['Sexo'].value_counts(normalize = True).round(4) * 100
dist_freq_sexo = pd.DataFrame({'Frequência': frequencia_sexo, 'Porcentagem (%)': percentual_sexo})
dist_freq_sexo.rename_axis('Sexo', axis = 'columns', inplace = True) 
dist_freq_sexo

## <font color=green>Distribuição de frequências para variáveis quantitativas e teste de nomalidade</font>
***

In [ ]:
ax = sns.boxplot(data = dados[["PAD BD", "PAD BE"]], orient = 'v')
ax.figure.set_size_inches(6, 3)
ax.set_title('Variáveis Quantitativas PAD Direito x Esquerdo', fontsize=14)
ax

In [ ]:
ax = sns.boxplot(data = dados[["PAS BD", "PAS BE"]], orient = 'v')
ax.figure.set_size_inches(6, 4)
ax.set_title('Variáveis Quantitativas PAS Direito x Esquerdo', fontsize=14)
ax

In [ ]:
ax = sns.boxplot(data = dados[["Comprimento BD Seg", "Comprimento BE Seg"]], orient = 'v')
ax.figure.set_size_inches(6, 4)
ax.set_title('Variáveis Quantitativas Comprimento Direito x Esquerdo', fontsize=14)
ax

In [ ]:
ax = sns.boxplot(data = dados[["Circunferencia BD", "Circunferencia BE"]], orient = 'v')
ax.figure.set_size_inches(6, 4)
ax.set_title('Variáveis Quantitativas Circunferência Direito x Esquerdo', fontsize=14)
ax

In [ ]:
ax = sns.boxplot(data = dados[["Triceps BD", "Triceps BE"]], orient = 'v')
ax.figure.set_size_inches(6, 4)
ax.set_title('Variáveis Quantitativas Tríceps Direito x Esquerdo', fontsize=14)
ax

In [ ]:
ax = sns.boxplot(data = dados[["Circunferencia Muscular BD", "Circunferencia Muscular BE"]], orient = 'v')
ax.figure.set_size_inches(6, 4)
ax.set_title('Variáveis Quantitativas Circunferência Muscular Direito x Esquerdo', fontsize=14)
ax

In [ ]:
from plotnine import *

In [ ]:
(
    ggplot(dados[['Indice de Gordura BD', 'Indice de Gordura BE']].melt(), aes(x="variable", y="value")) + 
    geom_boxplot()
)

### **Função para Teste de Normalidade**

In [ ]:
from scipy.stats import probplot
from scipy.stats import shapiro

In [ ]:
def teste_normalidade(dados):
    for i in dados.columns:
        stat_test, p_valor = normaltest(dados[[i]])
        if(p_valor <= 0.05):
            print(f" {i} - Distribuição NÃO normal") # Rejeita H0
        else:
            print(f" ************* {i} - Distribuição normal")
            
#t is based on D’Agostino and Pearson’s [1], [2] test that combines skew and kurtosis to produce an omnibus test of normality.
#1 D’Agostino, R. B. (1971), “An omnibus test of normality for moderate and large sample size”, Biometrika, 58, 341-348
#2 D’Agostino, R. and Pearson, E. S. (1973), “Tests for departure from normality”, Biometrika, 60, 613-622

In [ ]:
def teste_normalidade_shapiro(dados):
    for i in dados.columns:
        w_test, p_valor = shapiro(dados[[i]])
        if(p_valor <= 0.05):
            print(f" {i} - Distribuição NÃO normal com p-valor: {round(p_valor, 4)}") # Rejeita H0
        else:
            print(f" ************* {i} - Distribuição normal com p-valor: {round(p_valor, 4)}")
            

In [ ]:
def teste_normalidade_shapiro_log(dados):
    for i in dados.columns:
        w_test, p_valor = shapiro(np.log(dados[[i]]))
        if(p_valor <= 0.05):
            print(f" {i} - Distribuição NÃO normal com p-valor: {round(p_valor, 4)}") # Rejeita H0
        else:
            print(f" ************* {i} - Distribuição normal com p-valor: {round(p_valor, 4)}")
            

### **Teste de Normalidade Braço Direito**

In [ ]:
teste_normalidade_shapiro(dados_bd.drop(columns=['ID', 'Sexo']))

Com base no teste de Shapiro-Wilk não há evidências de que variável PAS, na sua escala original, seja normal, dado o baixo p-valor. Apesar disso, o seu comportamento é bastante semelhante a uma normal, dado que é simétrica e sem valores muito extremos.  
Por outro lado, aplicando uma transformação logarítmica, muito comum em variáveis contínuas e sempre positivas, passamos a não rejeitar a hipotese nula de normalidade para a variável PAS. 

In [ ]:
teste_normalidade_shapiro_log(dados_bd.drop(columns=['ID', 'Sexo']))

$$Y = \beta_{0} + \beta_{1}x_i$$

$$Y_{ijk} = \beta_{0} + \beta_{1}x_i + \delta_i + \delta_k$$

$$\log Y = \beta_0 + \beta_1 x_i$$

$$Y = e^{\beta_0 + \beta_1 x_i}$$

$$Y = (\beta_{0} + \beta_{1})^2$$

$$Y_i \sim N(\beta_{0}, \beta_{1}x_i, \sigma^2)$$

X = 4

$z_1$ = 1, se o indivíduo percenter ao grupo 1, 0 caso contrário  
$z_2$ = 1, se o indi                         2, 0 caso contrário  
$z_3$ = 1, se o indi                         3, 0 caso contrário   

Se $z_1 = z_2 = z_3 = 0$, então o indi  é do 4








### **Teste de Normalidade Braço Esquerdo**

In [ ]:
teste_normalidade_shapiro(dados_be.drop(columns=['ID', 'Sexo']))

In [ ]:
teste_normalidade_shapiro_log(dados_be.drop(columns=['ID', 'Sexo']))

### **Compração das médias dos hemicorpos**

Utilizaremos um teste não paramétrio para commpação das médias dos parametros dos hemicorpos. Mann-Whitney é um teste não paramétrico utilizado para verificar se duas amostras independentes foram selecionadas a partir de populações que têm a mesma média. 

### <font color='red'>Critério do valor $p$</font>

> ### Rejeitar $H_0$ se o valor $p\leq\alpha$ 

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind, ttest_rel
from scipy.stats import wilcoxon
significancia = 0.05

In [ ]:
def compara_medias_normal(dados1, dados2):
    p_list = list()
    p_compara = list()
    for i in dados1.columns:
        u, p_valor = ttest_ind(dados1[[i]], dados2[[i]])
        if p_valor[0] <= significancia:
            p_compara.append("Médias diferentes")
        else:
            p_compara.append("Médias iguais")
        p_list.append(p_valor[0])
    return p_list, p_compara

In [ ]:
def compara_medias(dados1, dados2):
    p_list = list()
    p_compara = list()
    for i in dados1.columns:
        u, p_valor = mannwhitneyu(dados1[[i]], dados2[[i]])
        if p_valor[0] <= significancia:
            p_compara.append("Médias diferentes")
        else:
            p_compara.append("Médias iguais")
        p_list.append(p_valor[0])
    return p_list, p_compara

In [ ]:
def compara_medias_wilcoxon(dados1, dados2):
    p_list = list()
    p_compara = list()
    for i in dados1.columns:
        u, p_valor = wilcoxon(dados1[i], dados2[i])
        if p_valor <= significancia:
            p_compara.append("Médias diferentes")
        else:
            p_compara.append("Médias iguais")
        p_list.append(p_valor)
    return p_list, p_compara

In [ ]:
def compara_medias_par_ttest(dados1, dados2):
    p_list = list()
    p_compara = list()
    for i in dados1.columns:
        u, p_valor = ttest_rel(dados1[i], dados2[i])
        if p_valor <= significancia:
            p_compara.append("Médias diferentes")
        else:
            p_compara.append("Médias iguais")
        p_list.append(p_valor)
    return p_list, p_compara

**TESTES INDEPENDENTES**

**Comparação de médias como distribuição não normal**

In [ ]:
medias_hemicorpo_bd = dados_bd.drop(columns=['ID', 'Sexo']).mean()
medias_hemicorpo_bd = pd.DataFrame(data=medias_hemicorpo_bd, columns=["Direito"])

medias_hemicorpo_be = dados_be.drop(columns=['ID', 'Sexo']).mean()
medias_hemicorpo_be = pd.DataFrame(data=medias_hemicorpo_be, columns=["Esquerdo"])

medias_hemicorpos = pd.concat([medias_hemicorpo_bd,medias_hemicorpo_be], axis=1)
medias_hemicorpos.round(2)

p_list, p_compara = compara_medias(dados_bd.drop(columns=['ID', 'Sexo']),dados_be.drop(columns=['ID', 'Sexo']))
medias_hemicorpos["p_value"] = p_list
medias_hemicorpos["p_compare"] = p_compara
medias_hemicorpos.round(3)


**Comparação de médias como distribuição normal**

In [ ]:
p_list, p_compara = compara_medias_normal(dados_bd.drop(columns=['ID', 'Sexo']),dados_be.drop(columns=['ID', 'Sexo']))
medias_hemicorpos["p_value"] = p_list
medias_hemicorpos["p_compare"] = p_compara
medias_hemicorpos.round(3)

**TESTES DEPENDENTES**

In [ ]:
p_list, p_compara = compara_medias_wilcoxon(dados_bd.drop(columns=['ID', 'Sexo']),dados_be.drop(columns=['ID', 'Sexo']))
medias_hemicorpos["p_value"] = p_list
medias_hemicorpos["p_compare"] = p_compara
medias_hemicorpos.round(3)

In [ ]:
p_list, p_compara = compara_medias_par_ttest(dados_bd.drop(columns=['ID', 'Sexo']),dados_be.drop(columns=['ID', 'Sexo']))
medias_hemicorpos["p_value"] = p_list
medias_hemicorpos["p_compare"] = p_compara
medias_hemicorpos.round(3)

# <font color='red' style='font-size: 30px;'> CORRELACAO</font>
<hr style='border: 2px solid red;'>

The Pearson method is used by default, but the Pandas allows the use of other indexes. Pearson’s correlation measures the linear association between continuous variables. It is the value that indicates how much the relationship between variables can be described by a line. Interpreting the value of ρ


    0.9 to 1 positive or negative indicates a very strong correlation.
    0.7 to 0.9 positive or negative indicates a strong correlation.
    0.5 to 0.7 positive or negative indicates a moderate correlation.
    0.3 to 0.5 positive or negative indicates a weak correlation.
    0 to 0.3 positive or negative indicates a negligible correlation.

Perhaps the most common correlation measure is Pearson’s correlation that assumes a Gaussian distribution to each variable and reports on their linear relationship.

### <font color=blue>Correlação para 978 dados</font>

**SpearMan**

In [ ]:
corr = dados_dobrados.corr(method="spearman")
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
_, ax = plt.subplots(figsize=(10, 7))
ax = sns.heatmap(corr, annot= True, vmin = -1, vmax = 1, square = True, mask=mask, center=0 )

In [ ]:
dados_dobrados.corr(method="spearman")['PAS']

In [ ]:
df = pd.DataFrame()
feat1s = []
feat2s = []
corrs = []
p_values = []
dados = dados_dobrados

for feat1 in dados:
    for feat2 in dados:
        if feat1 != feat2:
            feat1s.append(feat1)
            feat2s.append(feat2)
            corr, p_value = stats.spearmanr(dados[feat1], dados[feat2])
            corrs.append(corr)
            p_values.append(p_value)

df['Feature_1'] = feat1s
df['Feature_2'] = feat2s
df['Correlation'] = corrs
df['p_value'] = p_values
df.loc[df['Feature_1'] == 'PAS']

In [ ]:
dados_dobrados.to_pickle("dados_dobrados.pkl") ## Exportando tabela

### <font color=blue>Correlação Braço Direito</font>

In [ ]:
corr = dados_bd.corr(method="spearman")
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
_, ax = plt.subplots(figsize=(10, 7))
ax = sns.heatmap(corr, annot= True, vmin = -1, vmax = 1, square = True, mask=mask, center=0 )

In [ ]:
dados_bd.corr(method="spearman")['PAS'].round(3)

In [ ]:
df = pd.DataFrame()
feat1s = []
feat2s = []
corrs = []
p_values = []
dados = dados_bd

for feat1 in dados:
    for feat2 in dados:
        if feat1 != feat2:
            feat1s.append(feat1)
            feat2s.append(feat2)
            corr, p_value = stats.spearmanr(dados[feat1], dados[feat2])
            corrs.append(corr)
            p_values.append(p_value)

df['Feature_1'] = feat1s
df['Feature_2'] = feat2s
df['Correlation'] = corrs
df['p_value'] = p_values
df.loc[df['Feature_1'] == 'PAS']

### <font color=blue>Correlação para Braço Esquerdo</font>

In [ ]:
corr = dados_be.corr(method="spearman")
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
_, ax = plt.subplots(figsize=(10, 7))
ax = sns.heatmap(corr, annot= True, vmin = -1, vmax = 1, square = True, mask=mask, center=0 )

In [ ]:
dados_be.corr(method="spearman")['PAS'].round(3)

In [ ]:
df = pd.DataFrame()
feat1s = []
feat2s = []
corrs = []
p_values = []
dados = dados_be
for feat1 in dados:
    for feat2 in dados:
        if feat1 != feat2:
            feat1s.append(feat1)
            feat2s.append(feat2)
            corr, p_value = stats.spearmanr(dados[feat1], dados[feat2])
            corrs.append(corr)
            p_values.append(p_value)

df['Feature_1'] = feat1s
df['Feature_2'] = feat2s
df['Correlation'] = corrs
df['p_value'] = p_values
df.loc[df['Feature_1'] == 'PAS']